In [ ]:
# Check GPU
import torch
print('Torch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU name:', torch.cuda.get_device_name(0))
else:
    print('Enable GPU in Runtime > Change runtime type')


In [ ]:
# Option A: Mount Google Drive and use existing project in Drive (edit the path)
from google.colab import drive
import os

drive.mount('/content/drive', force_remount=True)

PROJECT_DIR = '/content/drive/MyDrive/your_project_path/genai_A1'  # <-- EDIT THIS
Q1_DIR = os.path.join(PROJECT_DIR, 'Q1')

%cd $Q1_DIR
os.environ['PYTHONPATH'] = Q1_DIR
!pwd && ls -la


In [ ]:
# Option B: Clone from GitHub (replace with your repo URL)
import os

%cd /content
!git clone https://github.com/your/repo.git genai_A1
%cd /content/genai_A1/Q1
os.environ['PYTHONPATH'] = '/content/genai_A1/Q1'
!pwd && ls -la


In [ ]:
# Option C: Upload a ZIP and unzip to /content
from google.colab import files
import os

uploaded = files.upload()  # upload your project zip
zip_name = list(uploaded.keys())[0]
!unzip -o "$zip_name" -d /content
%cd /content/genai_A1/Q1
os.environ['PYTHONPATH'] = '/content/genai_A1/Q1'
!pwd && ls -la


In [ ]:
# Install dependencies
%pip install -U pip
%pip install -r /content/genai_A1/requirements.txt

# Optional: Speed up HF datasets cache
import os
os.environ['HF_DATASETS_CACHE'] = '/content/hf_cache'
!mkdir -p /content/hf_cache


In [ ]:
# Ensure we are in Q1 dir and outputs exists
import os, pathlib
Q1_DIR = os.getcwd()
print('Working dir:', Q1_DIR)
os.makedirs('outputs', exist_ok=True)
!ls -la


In [ ]:
# ===============================================
# TASK 1: Dataset Preparation & Testing
# ===============================================
print("🚀 TASK 1: Testing Dataset Loading...")

# Test CIFAR-10 dataset loading from HuggingFace
!python -c "from src.dataset import CIFAR10HFDataset; ds = CIFAR10HFDataset(split='train[:100]'); print(f'✅ Dataset loaded! Shape: {ds[0][0].shape}, Label: {ds[0][1]}, Classes: 10')"

# Test model creation
!python -c "from src.model import SimpleCNN; model = SimpleCNN(); print(f'✅ Model created with {sum(p.numel() for p in model.parameters()):,} parameters')"

print("✅ TASK 1 COMPLETE: Dataset and model verified!")


In [ ]:
# ===============================================
# TASK 2: Build CNN and Train with Training Curves
# ===============================================
print("🧠 TASK 2: Training CNN with Training Curves...")

# Train baseline model (balanced epochs for good performance + reasonable time)
!python -m src.train --epochs 25 --batch_size 128 --lr 0.001 --num_layers 3 --base_filters 32 --outdir outputs

# Display training curve
from IPython.display import Image, display
print("📈 Training Loss Curve:")
display(Image('outputs/loss_curve.png'))

print("✅ TASK 2 COMPLETE: CNN trained with training curves!")


In [ ]:
# ===============================================
# TASK 3: Model Evaluation with Confusion Matrix & Metrics
# ===============================================
print("📊 TASK 3: Evaluating Model Performance...")

# Evaluate the trained model on test data
!python -m src.evaluate --model_path outputs/best_model.pt --batch_size 128 --num_layers 3 --base_filters 32 --outdir outputs

# Display results
from IPython.display import Image, display
import pandas as pd

print("🎯 Confusion Matrix:")
display(Image('outputs/confusion_matrix.png'))

print("📋 Performance Metrics:")
metrics = pd.read_csv('outputs/metrics.csv')
display(metrics)

print("📊 Per-Class Performance:")
per_class = pd.read_csv('outputs/per_class_metrics.csv')
display(per_class.head(10))

print("✅ TASK 3 COMPLETE: Model evaluated with confusion matrix and metrics!")


In [ ]:
# ===============================================
# TASK 4: Feature Map Visualization & Analysis
# ===============================================
print("🔍 TASK 4: Extracting and Visualizing Feature Maps...")

# Extract and visualize feature maps from different layers
!python -m src.visualize_features --model_path outputs/best_model.pt --batch_size 128 --num_layers 3 --base_filters 32 --outdir outputs --num_samples 6

# Display feature maps
from IPython.display import Image, display
import os

print("🎨 Feature Maps from Different Convolutional Layers:")
display(Image('outputs/feature_maps.png'))

print("🧠 Layer-wise Feature Analysis:")
if os.path.exists('outputs/feature_analysis.txt'):
    with open('outputs/feature_analysis.txt', 'r') as f:
        print(f.read())

print("✅ TASK 4 COMPLETE: Feature maps visualized and analyzed!")


In [ ]:
# ===============================================
# TASK 5: Hyperparameter Ablation Study
# ===============================================
print("⚡ TASK 5: Running Comprehensive Ablation Study...")
print("Testing: Learning Rate, Batch Size, Conv Filters, Number of Layers")

# Run ablation study (balanced epochs for thorough comparison)
!python -m src.ablation_study --outdir outputs --epochs 15

# Display results
import pandas as pd
import matplotlib.pyplot as plt

print("📊 Ablation Study Results:")
ablation_results = pd.read_csv('outputs/ablation_results.csv')
display(ablation_results)

print("🏆 Best Hyperparameter Configurations:")
if os.path.exists('outputs/best_ablation_configs.json'):
    import json
    with open('outputs/best_ablation_configs.json', 'r') as f:
        best_configs = json.load(f)
    for exp_type, config in best_configs.items():
        print(f"{exp_type}: {config}")

print("✅ TASK 5 COMPLETE: Ablation study finished!")


In [ ]:
# ===============================================
# TASK 6: Optimal Model Training & Performance Comparison
# ===============================================
print("🏆 TASK 6: Training Optimal Model and Comparing Performance...")

# Train model with optimal hyperparameters (more epochs for best performance)
!python -m src.train_optimal --epochs 30 --best_config_file outputs/best_ablation_configs.json --outdir outputs

# Evaluate optimal model
!python -m src.evaluate --model_path outputs/optimal_model.pt --batch_size 128 --outdir outputs

# Display comparison results
import pandas as pd
from IPython.display import Image, display

print("📈 Optimal Model Training Curves:")
if os.path.exists('outputs/optimal_training_curves.png'):
    display(Image('outputs/optimal_training_curves.png'))

print("⚖️ Model Performance Comparison:")
if os.path.exists('outputs/model_comparison.csv'):
    comparison = pd.read_csv('outputs/model_comparison.csv')
    display(comparison)

print("✅ TASK 6 COMPLETE: Optimal model trained and compared!")


In [ ]:
# ===============================================
# SUMMARY & RESULTS OVERVIEW
# ===============================================
print("📋 FINAL RESULTS SUMMARY")
print("="*50)

# List all generated files
print("📁 Generated Files:")
!ls -la outputs/

# Show key metrics
import pandas as pd
import os

if os.path.exists('outputs/metrics.csv'):
    print("\n🎯 Final Model Performance:")
    metrics = pd.read_csv('outputs/metrics.csv')
    display(metrics)

if os.path.exists('outputs/model_comparison.csv'):
    print("\n⚖️ Baseline vs Optimal Model:")
    comparison = pd.read_csv('outputs/model_comparison.csv')
    display(comparison)

if os.path.exists('outputs/ablation_results.csv'):
    print("\n📊 Best Hyperparameters Found:")
    ablation = pd.read_csv('outputs/ablation_results.csv')
    best_row = ablation.loc[ablation['accuracy'].idxmax()]
    print(f"Best Configuration: {best_row.to_dict()}")

print("\n🎉 ALL TASKS COMPLETED SUCCESSFULLY!")
print("✅ Dataset loaded and preprocessed")
print("✅ CNN trained with training curves")
print("✅ Model evaluated with confusion matrix")
print("✅ Feature maps visualized and analyzed")
print("✅ Hyperparameter ablation study completed")
print("✅ Optimal model trained and compared")


In [ ]:
# ===============================================
# DOWNLOAD ALL RESULTS - Q1_OUTPUTS
# ===============================================
print("📦 Preparing Q1_OUTPUTS for download...")

# Show what files we generated
print("📁 Generated Files:")
!ls -la outputs/

# Create single comprehensive ZIP file
print("\n🗜️ Creating Q1_OUTPUTS.zip...")
!zip -r /content/Q1_OUTPUTS.zip outputs/ -x "*.pyc" "*__pycache__*"

# Download the complete package
from google.colab import files
print("\n⬇️ Downloading Q1_OUTPUTS.zip to your local CPU...")
files.download('/content/Q1_OUTPUTS.zip')

print("\n✅ DOWNLOAD COMPLETE!")
print("📦 File downloaded: Q1_OUTPUTS.zip")
print("💻 Check your Downloads folder")

print("\n📋 Your Q1_OUTPUTS.zip contains:")
print("🔹 All model checkpoints (.pt files)")
print("🔹 Training curves and loss plots")
print("🔹 Confusion matrix visualization")
print("🔹 Performance metrics tables")
print("🔹 Feature map visualizations")
print("🔹 Complete ablation study results")
print("🔹 Model comparison data")
print("🔹 All generated analyses and plots")


In [ ]:
# ===============================================
# EXTRACT AND ANALYZE RESULTS LOCALLY
# ===============================================
print("📁 Instructions for using your Q1_OUTPUTS.zip:")
print("\n1. 📂 Go to your Downloads folder")
print("2. 🗜️ Extract Q1_OUTPUTS.zip")
print("3. 📊 Open the 'outputs' folder")
print("4. 🔍 Explore your results:")

print("\n   📈 Training Analysis:")
print("   - loss_curve.png - Training progress")
print("   - optimal_training_curves.png - Best model training")

print("\n   🎯 Model Performance:")
print("   - confusion_matrix.png - Classification results")
print("   - metrics.csv - Accuracy, precision, recall, F1")
print("   - per_class_metrics.csv - Per-class performance")

print("\n   🔬 Feature Analysis:")
print("   - feature_maps.png - What CNN layers learned")
print("   - feature_analysis.txt - Layer analysis")

print("\n   ⚡ Hyperparameter Study:")
print("   - ablation_results.csv - All tested configurations")
print("   - best_ablation_configs.json - Optimal settings")

print("\n   ⚖️ Model Comparison:")
print("   - model_comparison.csv - Baseline vs Optimal")

print("\n   🤖 Model Files:")
print("   - best_model.pt - Baseline trained model")
print("   - optimal_model.pt - Best hyperparameter model")

print("\n🎉 All 6 tasks completed successfully!")
print("🚀 Your complete Q1 analysis is ready for review!")
